In [23]:
import xml.etree.ElementTree as ET
import json

def extract_metadata(emxml_file, json_file):
    # Load JSON file
    with open(json_file, 'r') as f:
        metadata_mapping = json.load(f)

    # Load XML file
    tree = ET.parse(emxml_file)
    root = tree.getroot()

    metadata = {}

    # Extract acquisition metadata
    acquisition_metadata = metadata_mapping.get('acquisition')
    for key, value in acquisition_metadata.items():
        node = root.find(value)
        if node is not None:
            metadata[key] = node.text

    # Extract dataset metadata
    dataset_metadata = metadata_mapping.get('dataset')
    for key, value in dataset_metadata.items():
        node = root.find(value)
        if node is not None:
            metadata[key] = node.text

    # Extract image metadata
    image_metadata = metadata_mapping.get('image')
    for key, value in image_metadata.items():
        node = root.find(value)
        if node is not None:
            metadata[key] = node.text

    return metadata

# Usage example
json_file = '/Users/elias/Desktop/PP13_Mapping/pp13-mapper/schemas/sem_fib_nested_schema_map.json'
emxml_file = '/Users/elias/Desktop/NFDI Tomographiedaten/20200818_AlSi13 XRM tomo2/EMproject.emxml'
metadata = extract_metadata(emxml_file, json_file)
print(metadata)


{}


In [39]:
import json

def extract_EM_metadata(json_file):
    # Read the JSON file
    with open(json_file, 'r') as file:
        data = json.load(file)

    # Extract the "acquisition" information
    acquisition_data = data.get('acquisition', {})

    # Store key-value pairs starting with "EMproject"
    EM_metadata = {}
    for key, value in acquisition_data.items():
        if value.startswith('EMproject'):
            EM_metadata[key] = value

    return EM_metadata

emMetadata = extract_EM_metadata(json_file)


# Print each key-value pair in a new line
for key, value in emMetadata.items():
    print(key + ": " + value)

acquisition.genericMetadata.program.programName: EMproject.ApplicationName
acquisition.genericMetadata.program.programVersion: EMproject.ApplicationVersion
acquisition.genericMetadata.applicationId.identifierValue: EMproject.ApplicationId
acquisition.genericMetadata.fileVersion: EMproject.FileVersion
acquisition.genericMetadata.projectName: EMproject.ProjectName
acquisition.genericMetadata.zCutSpacing.value: EMproject.ZCutSpacing
acquisition.genericMetadata.numberOfCuts: EMproject.DataSets.Dataset.NumberOfCuts


In [87]:
import xml.etree.ElementTree as ET

def extract_values_from_emxml(EM_metadata, emxml_file):
    # Read the emxml file
    tree = ET.parse(emxml_file)
    root = tree.getroot()

    # Extract the namespace from the root element
    namespace = root.tag.split('}')[0] + '}'

    # Initialize dictionary to store extracted values
    extracted_values = {}

    # Extract values based on EM_metadata keys
    for key, value in EM_metadata.items():
        # Split the value by dot separator to get the hierarchy levels
        levels = value.split('.')
        element = root

        try:
            # Traverse through the hierarchy levels
            for level in levels:
                element = element.find(namespace + level)

                if element is None:
                    # If any level is not found, skip further processing for this key
                    raise Exception(f"Level '{level}' not found in XML")

            extracted_value = element.text if element is not None else None
            extracted_values[key] = extracted_value

        except Exception as e:
            print(f"Error extracting value for key '{key}': {str(e)}")

    return extracted_values


In [88]:
extract_values_from_emxml(emMetadata, emxml_file)

Error extracting value for key 'acquisition.genericMetadata.program.programName': Level 'EMproject' not found in XML
Error extracting value for key 'acquisition.genericMetadata.program.programVersion': Level 'EMproject' not found in XML
Error extracting value for key 'acquisition.genericMetadata.applicationId.identifierValue': Level 'EMproject' not found in XML
Error extracting value for key 'acquisition.genericMetadata.fileVersion': Level 'EMproject' not found in XML
Error extracting value for key 'acquisition.genericMetadata.projectName': Level 'EMproject' not found in XML
Error extracting value for key 'acquisition.genericMetadata.zCutSpacing.value': Level 'EMproject' not found in XML
Error extracting value for key 'acquisition.genericMetadata.numberOfCuts': Level 'EMproject' not found in XML


{}